# Advent of Code 2017

[see here](https://adventofcode.com/2017/)

## Preparation

import of some useful libs and definition of some common utility functions

In [1]:
# Python 3.x
import re
import numpy as np
import math
import urllib.request
import reprlib

from collections import Counter, defaultdict, namedtuple, deque
from functools   import lru_cache
from itertools   import permutations, combinations, chain, cycle, product, islice, count
from heapq       import heappop, heappush

def Input(day,strip=True):
    "Open this day's input file."
    
    filename = 'input/input{}.txt'.format(day)
    try:
        with open(filename, 'r') as f:
            text = f.read()
            if strip:
                text = text.strip()
        return text
    except FileNotFoundError:
        url = 'http://adventofcode.com/2017/day/{}/input'.format(day)
        print('input file not found. opening browser...')
        print('please save the file as "input<#day>.txt in your input folder.')
        import webbrowser
        webbrowser.open(url)

cat = ''.join
first = lambda x: list(x)[0]
def fs(*items): return frozenset(items)

def shift(it, n):
    return it[n:] + it[:n]

def rot(original,clockwise=True):
    '''rotate 2D matrix'''
    if clockwise:
        return list(zip(*original[::-1]))
    else:
        return list(zip(*original[::-1]))[::-1]

def locate2D(m, val):
    '''locate value in 2D list'''
    for i, line in enumerate(m):
        j=-1
        try:
            j = line.index(val)
        except ValueError:
            continue
        break
    else:
        i = -1
    return (i,j)

def dist_L1(p1,p2=(0,0)):
    return abs(p2[0]-p1[0])+abs(p2[1]-p1[1])

def dist_L2(p1,p2=(0,0)):
    return math.hypot(p2[0]-p1[0],p2[1]-p1[1])

def neighbors4(point): 
    "The four neighbors (without diagonals)."
    x, y = point
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1))

def neighbors8(point): 
    "The eight neighbors (with diagonals)."
    x, y = point 
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1),
            (x+1, y+1), (x-1, y-1), (x+1, y-1), (x-1, y+1))

#display and debug functions
def h1(s):
    return s + '\n' + '='*len(s) + '\n'

def trace1(f):
    "Print a trace of the input and output of a function on one line."
    rep = reprlib.aRepr
    rep.maxother = 85
    def traced_f(*args):
        arg_strs = ', '.join(map(rep.repr, args))
        result = f(*args)
        print('{}({}) = {}'.format(f.__name__, arg_strs, result))
        return result
    return traced_f

## Day 1

In [2]:
captcha = Input(1)
matching = (int(x) for x,y in zip(captcha,shift(captcha,1)) if x==y)
print(h1('Day 1 part 1 result: ' +str(sum(matching))))

Day 1 part 1 result: 995



In [3]:
matching = (int(x) for x,y in zip(captcha,shift(captcha,len(captcha)//2)) if x==y)
print(h1('Day 1 part 2 result: ' +str(sum(matching))))

Day 1 part 2 result: 1130



## Day 2

In [4]:
ss = Input(2)
numbers = [[int(x) for x in line.split('\t')] for line in ss.split('\n')]
checksum = sum(max(line)-min(line) for line in numbers)
print(h1('Checksum is '+str(checksum)))

Checksum is 48357



In [5]:
result = sum(first(int(max(x)/min(x)) for x in combinations(line,2) if max(x)%min(x) == 0) for line in numbers)
print(h1('Result is '+str(result)))

Result is 351



## Day 3

In [6]:
s = int(Input(3))
N = math.ceil(math.sqrt(s)) # minimum square size that contains s

def disp_spiral(sp):
    '''pretty-print spiral'''
    sp = list(sp)
    m = len(str(max(max(line) for line in sp)))
    print('\n'.join(' '.join('{: >{}}'.format(num,m) for num in line) for line in sp))
    
def gen_spiral(N):
    '''generate NxN spiral (highest number n^2)'''
    nums = [[j for j in range((i-1)*(i-1)+1,i*i+1)] for i in range(2,N+1)]
    square = [[1]]
    for num in nums:
        a, b = num[:len(square)],num[len(square):]
        square = rot(square) + [a]
        square = rot(square) + [b]
    return square

print(h1('Test Spiral for N=7:'))
disp_spiral(gen_spiral(7))

Test Spiral for N=7:

37 36 35 34 33 32 31
38 17 16 15 14 13 30
39 18  5  4  3 12 29
40 19  6  1  2 11 28
41 20  7  8  9 10 27
42 21 22 23 24 25 26
43 44 45 46 47 48 49


In [7]:
sp = gen_spiral(N)
d = dist_L1(locate2D(sp, 1),locate2D(sp, s))
print(h1('Part 1: ' + str(d) + ' steps needed'))

Part 1: 430 steps needed



In [8]:
ns = [[0 for _ in range(N)] for _ in range(N)] #generate zero filled square of same size
x,y = locate2D(sp,1) # use spiral as index
ns[x][y] = 1
i,n = 1,1
while n < s:
    i+=1
    x,y = locate2D(sp,i)
    neighbors = neighbors8((x,y))
    n = 0
    for xn, yn in neighbors:
        try: n += ns[xn][yn]
        except IndexError: pass
    ns[x][y] = n
print(h1('Part 2: first value written larger than the puzzle input (' + str(s) + ') is ' + str(n)))
print('The generated spiral:')
disp_spiral([line for line in rot([line for line in ns if any(line)]) if any(line)])

Part 2: first value written larger than the puzzle input (312051) is 312453

The generated spiral:
 17370  17008  16295  15252  14267  13486   6591      0
 35487    362    351    330    304    147   6444      0
 37402    747     11     10      5    142   6155      0
 39835    806     23      1      4    133   5733 312453
 42452    880     25      1      2    122   5336 295229
 45220    931     26     54     57     59   5022 279138
 47108    957   1968   2105   2275   2391   2450 266330
 48065  98098 103128 109476 116247 123363 128204 130654


## Day 4

In [9]:
passwords = Input(4).splitlines()
word = re.compile(r'\w+')
pass_words = {password: word.findall(password) for password in passwords}
valid = [pwd for pwd,words in pass_words.items() if len(words) == len(set(words))]
print(h1('Day 4 part 1: there are {} valid passwords.'.format(len(valid))))

Day 4 part 1: there are 325 valid passwords.



In [10]:
# sorting the words takes care of anagrams since all sorted anagrams are equal.
pass_words = {password: [cat(sorted(w)) for w in word.findall(password)] for password in passwords}
valid = [pwd for pwd,words in pass_words.items() if len(words) == len(set(words))]
print(h1('Day 4 part 2: there are {} valid passwords.'.format(len(valid))))

Day 4 part 2: there are 119 valid passwords.



## Day 5

In [11]:
jmp = list(map(int,Input(5).splitlines()))
def run(jmps, modfun=lambda x: x + 1):
    jmp = list(jmps)
    ip = 0 # instruction pointer
    steps = 0
    while ip < len(jmp):
        nip  = ip + jmp[ip]
        jmp[ip] = modfun(jmp[ip])
        ip = nip
        steps += 1
    return steps
assert run([0, 3, 0, 1, -3]) == 5 # example given in the 
steps = run(jmp)
print(h1('Day 5 part 1: escaped the maze after {} steps'.format(steps)))

Day 5 part 1: escaped the maze after 325922 steps



In [12]:
modfun = lambda x: x+1 if x < 3 else x-1
assert run([0, 3, 0, 1, -3], modfun) == 10 # example given in the 
steps = run(jmp, modfun)
print(h1('Day 5 part 2: escaped the maze after {} steps'.format(steps)))

Day 5 part 2: escaped the maze after 24490906 steps



## Day 6

In [13]:
s = tuple(map(int,Input(6).split()))
def distr(banks):
    banks = list(banks)
    i, v = max(enumerate(banks),key = lambda x: (x[1],-x[0]))
    banks[i] = 0
    for j in range(v): banks[(i + j + 1) % len(banks)] += 1
    return tuple(banks)

def realloc(banks):
    seen = set([banks])
    for it in count(1):
        banks = distr(banks)
        if banks in seen: return it, banks
        seen.add(banks)

assert realloc((0, 2, 7, 0)) == (5, (2, 4, 1, 2)), 'example from challenge desc'
assert realloc((4, 10, 4, 1, 8, 4, 9, 14, 5, 1, 14, 15, 0, 15, 3, 5))[0] == 12841, 'example from norvigs solution'
it, s = realloc(s)
print(h1('Day 6 part 1: infinite loop entered after {} steps'.format(it)))
it, s = realloc(distr(s))
print(h1('Day 6 part 2: length of infinite loop is {} steps'.format(it)))

Day 6 part 1: infinite loop entered after 6681 steps

Day 6 part 2: length of infinite loop is 2392 steps



## Day 7

In [14]:
nodes_text = Input(7)
test_text = '''
pbga (66)
xhth (57)
ebii (61)
havc (66)
ktlj (57)
fwft (72) -> ktlj, cntj, xhth
qoyq (66)
padx (45) -> pbga, havc, qoyq
tknk (41) -> ugml, padx, fwft
jptl (61)
ugml (68) -> gyxo, ebii, jptl
gyxo (61)
cntj (57)'''.strip()


class Node(object):
    def __init__(self, name, weight, children=tuple(), parent=None):
        self.name = name
        self.weight = int(weight)
        self.parent = parent
        self.children = set(children)
        
    def __repr__(self):
        chld = set(ch.name for ch in self.children)
        fstr = 'Node(name={name}, weight={weight}, children={chld}' + (", parent='{parent.name}'" if self.parent else '') +')'
        return fstr.format(chld=chld,**self.__dict__)
    
    @property
    def total_weight(self):
        return self.weight + sum(chld.total_weight for chld in self.children)
    

def set_parents(nodes):
    '''replace the name of the node.parent with the actual parent node'''
    for node in nodes.values():
        for chld in node.children:
            nodes[chld].parent = node
    return nodes

def set_children(nodes):
    '''replace the name of the node.children with the actual child nodes'''
    for node in nodes.values():
        node.children = set(nodes[chld] for chld in node.children)
    return nodes

def find_root(nodes):
    return first(node for node in nodes.values() if node.parent == None)

def populate_tree(text):
    '''build tree from input text'''
    split = re.compile(r'\w+').findall
    nodes = {n: Node(n, w, c)  for n,w,*c in map(split,text.splitlines())}
    nodes = set_parents(nodes)
    nodes = set_children(nodes)
    return nodes

assert find_root(populate_tree(test_text)).name == 'tknk', 'example from challenge description'

In [15]:
tree = populate_tree(nodes_text)
root = find_root(tree)
print(h1('Day 7 part 1: the root disk is "' + root.name +'".'))

Day 7 part 1: the root disk is "gmcrj".



In [16]:
def find_unbalanced(root_node):
    '''returns the node causing imbalance or the root node'''
    weights = {chld: chld.total_weight for chld in root_node.children}
    if len(set(weights.values())) == 1:
        return root_node
    cnt=Counter(w for _,w in weights.items())
    odd_weight = min(cnt,key=cnt.get)
    return find_unbalanced(first(node for node, w in weights.items() if w == odd_weight))

unbalanced = find_unbalanced(root)
balanced_weight = first(set(chld.total_weight for chld in unbalanced.parent.children) - {unbalanced.total_weight})
diff_to_other = balanced_weight - unbalanced.total_weight
print(h1('Day 7 part 2: the unbalanced disk is "{}", it should weight {} (currently {}).'.format(unbalanced.name, 
                                                                                              unbalanced.weight + diff_to_other,
                                                                                              unbalanced.weight
                                                                                              )))

Day 7 part 2: the unbalanced disk is "mdbtyw", it should weight 391 (currently 396).



## Day 8

In [17]:
def read_instr(text = None):
    if text == None:
        text = Input(8)
    instr = text.splitlines()
    # "translate" expressions
    Instruction = namedtuple('Instruction', 'dest, op, param, cmp1, cmp_op, cmp2')
    instr = [Instruction(*inst.replace(' if','').split(' ')) for inst in instr]
    return instr

def gen_cmp_ops(cmps):
    '''find the int-functions that implement the needed comparisions'''
    # get all int dunder-functions and their docs
    op_docs = {eval('int.'+f): eval('int.'+f+'.__doc__') for f in dir(int) if f.startswith('__')}
    # find int functions where their docs include the needed comparision
    cmp_ops = {cmp: fun for cmp, (fun, doc) in product(cmps,op_docs.items()) if doc != None and 'self'+cmp+'value' in doc}
    return cmp_ops

def exec_instr(registers, inst, cmp_ops):
    cmp1, cmp2 = expr(registers,inst.cmp1), expr(registers,inst.cmp2)
    op1, op2 = expr(registers,inst.dest), expr(registers,inst.param)
    if cmp_ops[inst.cmp_op](cmp1, cmp2):
        registers[inst.dest] = num_op[inst.op](op1, op2)
    return registers
    
# create register dict and expression lookup
expr = lambda registers, e: registers[e] if e.isalpha() else int(e)
# create numerical operation dict
num_op = {'inc': int.__add__,
          'dec': int.__sub__,
         }

def run(text=None):
    registers = defaultdict(int)
    instructions = read_instr(text)
    cmp_ops = gen_cmp_ops({i.cmp_op for i in instructions})
    max_all = 0
    for inst in instructions:
        registers = exec_instr(registers, inst, cmp_ops)
        max_all = max(max_all, max(registers.values()))
    return registers, max_all

test_inst = '''b inc 5 if a > 1
a inc 1 if b < 5
c dec -10 if a >= 1
c inc -20 if c == 10'''

assert max(run(test_inst)[0].values()) == 1

reg, max_all = run()
print(h1('Day 8 part 1: highest value in any register (post execution) is ' + str(max(reg.values()))))
print(h1('Day 8 part 2: highest value in any register (during execution) is ' + str(max_all)))

Day 8 part 1: highest value in any register (post execution) is 4877

Day 8 part 2: highest value in any register (during execution) is 5471



## Day 9

In [ ]:
data = Input(9)